In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
app_df = pd.read_csv("charity_data.csv")
app_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = app_df.drop(columns=['EIN','NAME'])

In [3]:
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_types = df['APPLICATION_TYPE'].value_counts()
application_types

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = [index for index,row in application_types.items() if row < 500]

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HEREclassification_types = df['CLASSIFICATION'].value_counts()
classification_types = df['CLASSIFICATION'].value_counts()
classification_types

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
[[index,row] for index,row in classification_types.items() if row > 1]

[['C1000', 17326],
 ['C2000', 6074],
 ['C1200', 4837],
 ['C3000', 1918],
 ['C2100', 1883],
 ['C7000', 777],
 ['C1700', 287],
 ['C4000', 194],
 ['C5000', 116],
 ['C1270', 114],
 ['C2700', 104],
 ['C2800', 95],
 ['C7100', 75],
 ['C1300', 58],
 ['C1280', 50],
 ['C1230', 36],
 ['C1400', 34],
 ['C7200', 32],
 ['C2300', 32],
 ['C1240', 30],
 ['C8000', 20],
 ['C7120', 18],
 ['C1500', 16],
 ['C1800', 15],
 ['C6000', 15],
 ['C1250', 14],
 ['C8200', 11],
 ['C1238', 10],
 ['C1278', 10],
 ['C1235', 9],
 ['C1237', 9],
 ['C7210', 7],
 ['C2400', 6],
 ['C1720', 6],
 ['C4100', 6],
 ['C1257', 5],
 ['C1600', 5],
 ['C1260', 3],
 ['C2710', 3],
 ['C0', 3],
 ['C3200', 2],
 ['C1234', 2],
 ['C1246', 2],
 ['C1267', 2],
 ['C1256', 2]]

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = [index for index,row in classification_types.items() if row < classification_types.mean()]

# Replace in dataframe
for cls in classifications_to_replace:
    app_df['CLASSIFICATION'] = app_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
app_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(df[['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']])
processed_df = pd.concat([df[['STATUS', 'ASK_AMT']], dummies], axis=1)

In [11]:
# Split our preprocessed data into our features and target arrays
X = processed_df
y = df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=.2,
                                                    random_state=42)

print(f"X_train Shape: {X_train.shape}, X_test Shape: {X_test.shape}")

X_train Shape: (27439, 108), X_test Shape: (6860, 108)


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=5, activation="tanh", input_dim=X_train_scaled.shape[1]))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=3, activation='tanh'))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 545       
                                                                 
 dense_1 (Dense)             (None, 3)                 18        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 567 (2.21 KB)
Trainable params: 567 (2.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
model.compile(loss="BinaryCrossentropy", optimizer="adam", metrics=['accuracy', 'mse'])

In [15]:
# Train the model
fit_model = model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))

Epoch 1/100
858/858 [==============================] - 4s 3ms/step - loss: 0.5895 - accuracy: 0.7085 - mse: 0.2009 - val_loss: 0.5726 - val_accuracy: 0.7182 - val_mse: 0.1934
Epoch 2/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5631 - accuracy: 0.7256 - mse: 0.1895 - val_loss: 0.5678 - val_accuracy: 0.7208 - val_mse: 0.1913
Epoch 3/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5590 - accuracy: 0.7283 - mse: 0.1878 - val_loss: 0.5664 - val_accuracy: 0.7188 - val_mse: 0.1910
Epoch 4/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5569 - accuracy: 0.7271 - mse: 0.1870 - val_loss: 0.5646 - val_accuracy: 0.7206 - val_mse: 0.1902
Epoch 5/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5557 - accuracy: 0.7280 - mse: 0.1866 - val_loss: 0.5635 - val_accuracy: 0.7214 - val_mse: 0.1898
Epoch 6/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5546 - accuracy: 0.7299 - mse: 0.1862 - val_loss

858/858 [==============================] - 3s 4ms/step - loss: 0.5432 - accuracy: 0.7346 - mse: 0.1822 - val_loss: 0.5563 - val_accuracy: 0.7281 - val_mse: 0.1872
Epoch 48/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5432 - accuracy: 0.7349 - mse: 0.1823 - val_loss: 0.5554 - val_accuracy: 0.7277 - val_mse: 0.1869
Epoch 49/100
858/858 [==============================] - 2s 3ms/step - loss: 0.5432 - accuracy: 0.7345 - mse: 0.1823 - val_loss: 0.5555 - val_accuracy: 0.7278 - val_mse: 0.1870
Epoch 50/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5429 - accuracy: 0.7348 - mse: 0.1822 - val_loss: 0.5561 - val_accuracy: 0.7277 - val_mse: 0.1871
Epoch 51/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5431 - accuracy: 0.7350 - mse: 0.1823 - val_loss: 0.5553 - val_accuracy: 0.7271 - val_mse: 0.1869
Epoch 52/100
858/858 [==============================] - 2s 3ms/step - loss: 0.5432 - accuracy: 0.7351 - mse: 0.1823 - val_loss: 0.555

858/858 [==============================] - 2s 2ms/step - loss: 0.5415 - accuracy: 0.7350 - mse: 0.1817 - val_loss: 0.5539 - val_accuracy: 0.7287 - val_mse: 0.1863
Epoch 94/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5412 - accuracy: 0.7358 - mse: 0.1815 - val_loss: 0.5542 - val_accuracy: 0.7289 - val_mse: 0.1864
Epoch 95/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5413 - accuracy: 0.7354 - mse: 0.1815 - val_loss: 0.5538 - val_accuracy: 0.7294 - val_mse: 0.1862
Epoch 96/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5412 - accuracy: 0.7347 - mse: 0.1815 - val_loss: 0.5544 - val_accuracy: 0.7289 - val_mse: 0.1865
Epoch 97/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5412 - accuracy: 0.7345 - mse: 0.1815 - val_loss: 0.5545 - val_accuracy: 0.7296 - val_mse: 0.1865
Epoch 98/100
858/858 [==============================] - 2s 3ms/step - loss: 0.5413 - accuracy: 0.7353 - mse: 0.1816 - val_loss: 0.554

In [16]:
# Evaluate the model using the test data
model.evaluate(X_test_scaled, y_test)

215/215 [==============================] - 0s 2ms/step - loss: 0.5538 - accuracy: 0.7283 - mse: 0.1863


[0.553793728351593, 0.7282798886299133, 0.1862620860338211]

In [17]:
# Export our model to HDF5 file
model.save("AlphabetSoupCharity_Optimization2.h5")

C:\Users\wnaiy\anaconda3\envs\ml\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
